<a href="https://colab.research.google.com/github/likeaAI/pytorch/blob/main/test02_cloths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from glob import glob

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="V1xrnYqaT6VW3skv3nh8")
project = rf.workspace("example01").project("test02-l7ug9")
dataset = project.version(1).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 4.1 MB/s 
     |████████████████████████████████| 178 kB 30.2 MB/s 
     |████████████████████████████████| 1.1 MB 62.9 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 138 kB 61.2 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.4-py3-none-any.whl size=21356 sha256=37c599a516d5302be25ba106e7fbdb939613412bf138c23699cc7a3453eddda2
  Stored in directory: /root/.cache/pip/wheels/b7/2c/60/3733c0e323b1d7aec00f852264314bf6c48192c8e1dc92b3ee
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=9ed3480cedc1a82f7de6afd4cc6c6de0858d0fc5a4ec19815fa501eee30ba60b
  Stored in directory: /root/.cache/pip/

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to test02-1 in yolov5pytorch:: 100%|██████████| 778/778 [00:00<00:00, 1863.18it/s]


In [3]:
!nvidia-smi

Tue May 31 12:23:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# yolov5 다운로드 
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 12142, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12142 (delta 2), reused 10 (delta 1), pack-reused 12129
Receiving objects: 100% (12142/12142), 11.93 MiB | 11.70 MiB/s, done.
Resolving deltas: 100% (8402/8402), done.


In [18]:
# 이미지 list 
train_img_list = glob('/content/test02-1/train/images/*.jpg')
val_img_list = glob('/content/test02-1/valid/images/*.jpg')
test_img_list = glob('/content/test02-1/test/images/*.jpg')

# label list 
train_label_list = glob('/content/test02-1/train/labels/*.txt')
val_label_list = glob('/content/test02-1/valid/labels/*.txt')
test_label_list = glob('/content/test02-1/test/labels/*.txt')

print(len(train_img_list) , len(val_img_list) , len(test_img_list))
print(len(train_label_list) , len(val_label_list) , len(test_label_list))

269 77 37
269 77 37


In [25]:

with open('/content/yolov5/test02-1/train.txt' , 'w') as f : 
    f.write('\n'.join(train_img_list))
with open('/content/yolov5/test02-1/val.txt' , 'w') as f : 
    f.write('\n'.join(val_img_list))
with open('/content/yolov5/test02-1/test.txt' , 'w') as f : 
    f.write('\n'.join(test_img_list))

In [24]:
import yaml

with open('/content/yolov5/test02-1/data.yaml' , 'r') as f : 
    data = yaml.safe_load(f) # yaml.load() x => yaml.safe_load(f)


data['train'] = '/content/yolov5/test02-1/train.txt'   
data['val'] = '/content/yolov5/test02-1/val.txt'
data['test'] = '/content/yolov5/test02-1/test.txt'

with open ('/content/yolov5/test02-1/data.yaml' , 'w') as f : 
    yaml.dump(data, f )

print(data)

{'names': ['jean', 'slack', 'top_long', 'top_short'], 'nc': 4, 'train': '/content/yolov5/test02-1/train.txt', 'val': '/content/yolov5/test02-1/val.txt', 'test': '/content/yolov5/test02-1/test.txt'}


In [27]:
%cat /content/yolov5/test02-1/data.yaml

names:
- jean
- slack
- top_long
- top_short
nc: 4
test: /content/yolov5/test02-1/test.txt
train: /content/yolov5/test02-1/train.txt
val: /content/yolov5/test02-1/val.txt


In [28]:
# 라벨인식을 못하는 문제 발생함 !!!! 확인할것. 
%cd /content/yolov5/

!python train.py --img 120 --batch 16 --epochs 50 \
--data /content/yolov5/test02-1/data.yaml \
--cfg ./models/yolov5s.yaml \
--weights yolov5s.pt \
--name test02_result

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/yolov5/test02-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=120, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=test02_result, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-240-g6dd6aea Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, 

In [ ]:
# 추론하기 

from IPython.display import Image
import os

val_img_path = val_img_list[0]

!python detect.py --weights /content/yolov5/runs/train/test02_result/weights/best.pt \

detect: weights=['/content/yolov5/runs/train/test02_result/weights/best.pt'], source=data/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-236-gdcf8073 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Traceback (most recent call last):
  File "detect.py", line 252, in <module>
    main(opt)
  File "detect.py", line 247, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "detect.py", line 92, in run
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
  File "/content/yolo